In [35]:
%load_ext autoreload
%autoreload 2
import dateutil
import json
from pyspark.sql import SQLContext, Row

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import sys
import inspect

currentdir = os.path.dirname(
    os.path.abspath(inspect.getfile(inspect.currentframe()))
)
parentdir = os.path.dirname(currentdir)
gpparentdir = os.path.dirname(parentdir)
sys.path.insert(0, gpparentdir)

from src.traces.trace_utils import *

In [69]:
sqlContext = SQLContext(sc)
table = 'test'
in_dir = '/user/ellery/readers'
out_dir = '/home/ellery/readers/data'

In [40]:
day = '2016-02-17'
host = 'en.wikipedia.org'

In [41]:
fname = get_file_name(day, host, table)

In [42]:
data_table = os.path.join(in_dir, fname )

In [43]:
trace_rdd = sc.textFile(data_table) \
            .map(lambda x: json.loads(x)) \
            .filter(lambda x: len(x) == 3) \
            .map(lambda x: Row(key=x['ip'] + x['ua'], requests=x['requests'])) \
            

traceDF = sqlContext.createDataFrame(trace_rdd)
traceDF.registerTempTable("traceDF")

In [54]:
clickDF = get_click_df(sc,sqlContext, day,host, 'clickDF')

In [56]:
query = """
SELECT *
FROM 
    traceDF JOIN clickDF
WHERE clickDF.key = traceDF.key
"""

res = sqlContext.sql(query).collect()

In [59]:
sqlContext.sql("SELECT COUNT(*) FROM clickDF ").collect()

[Row(_c0=12)]

In [57]:
len(res)

12

In [60]:
type(res)

list

In [67]:
click_traces = []
for row in res:
    d = {'key': row.key, 'requests': row.requests, 'click_data':row.click_data}
    click_traces.append(d)

In [71]:
path = os.path.join(out_dir, fname)
outfile = os.path.join(path, 'data.json')
os.makedirs( path )
json.dump(click_traces, open(outfile, 'w'))